### Imports

In [1]:
import ast
import pickle
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.decomposition import PCA

/home/dmac/.local/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### Model Definition

In [2]:
class DocModel(nn.Module):
    def __init__(self, input_size, hidden_sizes):
        super(DocModel, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        
        modlist = []
        for i in range(len(hidden_sizes)):
            if i == 0:
                modlist.append(nn.Sequential(
                    nn.Linear(self.input_size, self.hidden_sizes[i]),
                    nn.LeakyReLU()))
            else:
                modlist.append(nn.Sequential(
                    nn.Linear(self.hidden_sizes[i-1], self.hidden_sizes[i]),
                    nn.LeakyReLU()))
                               
        self.docvecpipeline = nn.ModuleList(modlist)
        
        self.regressor = nn.Sequential(
            nn.Linear(self.hidden_sizes[-1], 1)
        )
        
    def forward(self, invecs):
        hidden = invecs
        for i, layer in enumerate(self.docvecpipeline):
            hidden = layer(hidden)
        output = self.regressor(hidden)
        return output

### Define model hyperparams

In [3]:
dhs_clusts = pd.read_csv('dhs_metadata_modified_wikis_np.csv')
countries = list(dhs_clusts['country'].unique())
wikis = {country : pd.read_csv(f'articles/{country}_Wiki.csv') for country in countries}
for country, country_wiki in wikis.items():
    country_wiki['embedding'] = country_wiki['embedding'].apply(lambda x: np.fromstring(x[1:-1], 
                                                                                        sep=' '))
    country_wiki['w2vec'] = country_wiki['w2vec'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

In [4]:
dhs_folds = pickle.load(open("dhs_incountry_folds.pkl", "rb" ))

In [5]:
def get_article_embedds_from_idxs(row, wikis):
    '''
    Extract article features for a row of the DHS survey data, given some global
    curr_country representing the current country and a corresponding article dataframe.
    '''
    country = row['country']
    country_wiki = wikis[country]
    embedds = []
    w2vecs = []
    for i, closest_idx in enumerate(row['article_closest_idxs']):
        embedds.append(country_wiki['embedding'].iloc[int(closest_idx)])
        w2vecs.append(country_wiki['w2vec'].iloc[int(closest_idx)])
    embedds = torch.flatten(torch.tensor(np.array(embedds))).float().numpy()
    w2vecs = torch.flatten(torch.tensor(np.array(w2vecs))).float().numpy()
    
    return embedds, w2vecs

In [6]:
dhs_clusts['article_closest_idxs'] = dhs_clusts['article_closest_idxs'].apply(lambda x: np.fromstring(x[1:-1], 
                                                                                        sep=' '))
dhs_clusts['article_dists'] = dhs_clusts['article_dists'].apply(lambda x: np.fromstring(x[1:-1], 
                                                                                        sep=' '))
dhs_clusts['article_log2_lens'] = dhs_clusts['article_log2_lens'].apply(lambda x: np.fromstring(x[1:-1], 
                                                                                        sep=' '))
dhs_clusts[['article_embeddings','w2vec']] = dhs_clusts.apply(lambda x: get_article_embedds_from_idxs(x, wikis), 
                                                              axis=1, result_type='expand')

In [7]:
pca = PCA(n_components=1000)

In [8]:
pca.fit(dhs_clusts['article_embeddings'].values.tolist())

PCA(n_components=1000)

In [ ]:
dhs_clusts['article_embeddings_proj'] = dhs_clusts['article_embeddings'].progress_apply(lambda x : pca.transform([x])[0])

 52%|█████▏    | 10222/19669 [04:33<04:14, 37.15it/s]

In [ ]:
def get_tensor_inputs(row):
    return torch.cat([torch.tensor([row['year']- 2009]),
                      torch.tensor([row['lat']]),
                      torch.tensor([row['lon']]), 
                      torch.tensor(row['article_dists']),
                      torch.tensor(row['article_log2_lens']),
                      torch.tensor([row['num_near_articles']]), 
                      torch.tensor(row['article_embeddings_proj']),
                      torch.tensor(row['w2vec'])]).float()

In [ ]:
dhs_clusts['tensor_input'] = dhs_clusts.apply(get_tensor_inputs, axis=1)

### Train

In [ ]:
embed_size = 768
num_close_arts = 10

DocModelNet = DocModel(dhs_clusts['tensor_input'].iloc[0].shape[0], 
                       [256*2, 256*2, 128*2, 64*2, 16*2])

lr = 0.0001
criterion = nn.MSELoss()

optimizer = optim.Adam(DocModelNet.parameters(), lr=lr)

num_epochs = 25

device = ("cuda" if torch.cuda.is_available() else "cpu")

DocModelNet.to(device)

In [ ]:
fold = 'A'
train_idxs = dhs_folds[fold]['train']
val_idxs = dhs_folds[fold]['val']
test_idxs = dhs_folds[fold]['test']

In [ ]:
# avg loss per epoch
train_epoch_losses = []
val_epoch_losses = []

# Training Loop
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # list of individ cluster losses for this epoch
    train_iter_losses = []
    val_iter_losses = []
    # for each training point
    for i in range(len(train_idxs)):
        idx = train_idxs[i]
        # clear gradient fro prev instance
        DocModelNet.zero_grad()
        # Get inputs
        embedd = dhs_clusts['tensor_input'].iloc[idx].to(device)
        # get target wealth value 
        target = torch.tensor(dhs_clusts['wealthpooled'].iloc[idx]).float().to(device)
        # get model output
        output = DocModelNet(embedd)
        # Calculate loss based on this output and the loss funct
        err = criterion(output[0], target)
        # Calculate gradients
        err.backward()
        # Update network params using the optimizer
        optimizer.step()
        # Save Loss for plotting and analysis
        train_iter_losses.append(err.item())
    
    # Evalaute model (but don't backpropagate!) on validation set
    for i in range(len(val_idxs)):
        idx = val_idxs[i]
        # Get inputs
        embedd = dhs_clusts['tensor_input'].iloc[idx].to(device)
        # get target wealth value 
        target = torch.tensor(dhs_clusts['wealthpooled'].iloc[idx]).float().to(device)
        # get model output
        output = DocModelNet(embedd)
        # Calculate loss based on this output and the loss funct
        err = criterion(output[0], target)
        # Save Loss for plotting and analysis
        val_iter_losses.append(err.item())

    train_epoch_losses.append(np.mean(train_iter_losses))
    val_epoch_losses.append(np.mean(val_iter_losses))
    print(f"epoch {epoch}   avg train loss {train_epoch_losses[epoch]}   avg val loss {val_epoch_losses[epoch]}\n")

In [ ]:
def get_pred_wealth(clust, model, country_wiki):
    embedd = clust['tensor_input'].to(device)
    output = model(embedd)
    return output.item()

In [ ]:
dhs_clusts[fold + '_pred_wealth'] = \
dhs_clusts.apply(lambda clust : get_pred_wealth(clust, DocModelNet, country_wiki), axis=1)

In [ ]:
lin_model = Ridge(alpha=0)
X = dhs_clusts.iloc[test_idxs][fold + '_pred_wealth'].values.reshape(-1, 1)
y = dhs_clusts.iloc[test_idxs]['wealthpooled'].values
lin_model.fit(X, y)

print(f'R^2 = {lin_model.score(X, y):.5f}')
#r, p = ss.pearsonr(X[:,0], y)
#print('pearson r^2: ', r**2)

In [ ]:
#country_clusts['pred_wealth'].to_csv(f'early_{country}_({year})_preds.csv')

In [ ]:
fig, axs = plt.subplots(3,1,figsize=(10,30))

plt.subplot(311)
plt.title("DHS Wealth Indices vs Predicted Train Set")

plt.plot(dhs_clusts.iloc[train_idxs][fold + '_pred_wealth'].values, 
         dhs_clusts.iloc[train_idxs]['wealthpooled'].values, 'bo', alpha=0.5);

lin_model = Ridge(alpha=0)
X = dhs_clusts.iloc[train_idxs][fold + '_pred_wealth'].values.reshape(-1, 1)
y = dhs_clusts.iloc[train_idxs]['wealthpooled'].values
lin_model.fit(X, y)
R = f"{lin_model.score(X, y):.3f}"
plt.plot(dhs_clusts.iloc[train_idxs][fold + '_pred_wealth'].values,
        lin_model.predict(dhs_clusts.iloc[train_idxs][fold + '_pred_wealth'].values.reshape(-1, 1)),
        'k-', alpha=0.9, label=r'Train $R^2=$'+R)
plt.xlabel("Predicted Wealth Index")
plt.ylabel("Actual Wealth Index")
plt.legend()


plt.subplot(312)
plt.title("DHS Wealth Indices vs Predicted Validation Set")
plt.plot(dhs_clusts.iloc[val_idxs][fold + '_pred_wealth'].values, 
         dhs_clusts.iloc[val_idxs]['wealthpooled'].values, 'bo', alpha=0.5);
lin_model = Ridge(alpha=0)
X = dhs_clusts.iloc[val_idxs][fold + '_pred_wealth'].values.reshape(-1, 1)
y = dhs_clusts.iloc[val_idxs]['wealthpooled'].values
lin_model.fit(X, y)
R = f"{lin_model.score(X, y):.3f}"
plt.plot(dhs_clusts.iloc[val_idxs][fold + '_pred_wealth'].values,
        lin_model.predict(dhs_clusts.iloc[val_idxs][fold + '_pred_wealth'].values.reshape(-1, 1)),
        'k-', alpha=0.9, label=r'Val $R^2=$'+R)
plt.xlabel("Predicted Wealth Index")
plt.ylabel("Actual Wealth Index")
plt.legend()


plt.subplot(313)
plt.title("DHS Wealth Indices vs Predicted Test Set")
plt.plot(dhs_clusts.iloc[test_idxs][fold + '_pred_wealth'].values, 
         dhs_clusts.iloc[test_idxs]['wealthpooled'].values, 'bo', alpha=0.5);
lin_model = Ridge(alpha=0)
X = dhs_clusts.iloc[test_idxs][fold + '_pred_wealth'].values.reshape(-1, 1)
y = dhs_clusts.iloc[test_idxs]['wealthpooled'].values
lin_model.fit(X, y)
R = f"{lin_model.score(X, y):.3f}"
plt.plot(dhs_clusts.iloc[test_idxs][fold + '_pred_wealth'].values,
        lin_model.predict(dhs_clusts.iloc[test_idxs][fold + '_pred_wealth'].values.reshape(-1, 1)),
        'k-', alpha=0.9, label=r'Test $R^2=$'+R)
plt.xlabel("Predicted Wealth Index")
plt.ylabel("Actual Wealth Index")
plt.legend()

plt.savefig(f'{fold}_preds.png')
plt.show()

In [ ]:
#dhs_clusts.drop(columns=['article_embeddings', 'tensor_input']).to_csv('dhs_metadata_modified_wikis_np.csv', index=False)

In [ ]:
# # Optionally save the model

# save_model = False
# checkpoint_path = f"./models/articles/{fold}"
# if save_model:
#     torch.save({
#             'epoch': epoch,
#             'model_state_dict': DocModelNet.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict()
#             }, checkpoint_path)

In [ ]:
percentiles = dhs_clusts['wealthpooled'].quantile(q=np.arange(0.05, 1.05, 0.05)).values

In [ ]:
rs = []
for percentile in percentiles:
    lin_model = Ridge(alpha=0)
    X = dhs_clusts.iloc[test_idxs][dhs_clusts.iloc[test_idxs]['wealthpooled'] <= percentile][fold + '_pred_wealth'].values.reshape(-1, 1)
    y = dhs_clusts.iloc[test_idxs][dhs_clusts.iloc[test_idxs]['wealthpooled'] <= percentile]['wealthpooled'].values
    lin_model.fit(X, y)

    #print(f'R^2 = {lin_model.score(X, y):.5f}')
    rs.append(lin_model.score(X, y))
    
fig, axs = plt.subplots(1,1,figsize=(10,10))
axs.plot(np.arange(0.05, 1.05, 0.05), rs, '-o');
plt.xlabel("Percentile Wealth Index")
plt.ylabel(r"Pearson $r^2$ of Actual and Predicted Wealth")
plt.title("Model Performance at Different Wealth Levels");
plt.savefig(f'{fold}_percentile_preds.png')
#r, p = ss.pearsonr(X[:,0], y)
#print('pearson r^2: ', r**2)

### Compare with other Country

#### Load in second country data

In [ ]:
country2 = 'malawi'
country2_abrev = 'MWI'
year_country2 = [2014, 2015]

#### Create input features

In [ ]:
country2_clusts['pred_wealth'] = \
country2_clusts.apply(lambda clust : get_pred_wealth(clust, DocModelNet, country2_wiki), axis=1)

In [ ]:
lin_model = Ridge(alpha=0)
X = country2_clusts['pred_wealth'].values.reshape(-1, 1)
y = country2_clusts['wealthpooled'].values
lin_model.fit(X, y)

print('R^2 with no regularization (alpha=0): ', lin_model.score(X, y))

fold_n = 10
kf = KFold(n_splits=fold_n)
Rs = []
for train, test in kf.split(X, y):
    clf = Ridge(alpha=0.5).fit(X[train], y[train])
    Rs.append(clf.score(X[test], y[test]))
Rs = np.array(Rs)
print('Avg cross-validated R^2 with 10 folds and alpha = 0.5:', Rs.mean())
print('Standard Deviation of R^2 measurements:', Rs.std())

In [ ]:
country2_clusts['pred_wealth'].to_csv(f'early_{country}_({year})_to_{country2}_({year_country2})_preds.csv')

In [ ]:
plt.figure(figsize=(10,10))
country2_cap = country2[0].upper() + country2[1:]
plt.title(f"{country2_cap} {year_country2} DHS Wealth Indices vs Predicted From Model Trained on {country_cap} {year} Data")
plt.plot(country2_clusts['pred_wealth'].values, 
         country2_clusts['wealthpooled'].values, 'bo', alpha=0.7);
R = f"{lin_model.score(X, y):.4f}"
plt.plot(country2_clusts['pred_wealth'].values,
        lin_model.predict(country2_clusts['pred_wealth'].values.reshape(-1, 1)),
        'k-', alpha=0.7, label=r'$R^2=$'+R)
plt.xlabel("Predicted Wealth Index")
plt.ylabel("Actual Wealth Index")
plt.legend()
plt.savefig(f'early_{country}_({year})_to_{country2}_({year_country2})_preds.png')
plt.show()